<a href="https://colab.research.google.com/github/SUNNY11286/solar_with_ai_preds/blob/master/Bidirectional_LSTM_multivariate_horizon_style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evalution
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

import tensorflow as tf

In [54]:
data = pd.read_csv('out_clean.csv')
data

,Date,Total Generation (KWH),Insolation,Grid Failure,Inverter,Module Cleaning,Rainy day,No Module Cleaning,Transformer replacement and maintenance,Cable and Fuse maintenance,Plant Shutdown,Internet,Battery,Cloudy,Module Cleaning by rains,NO-OFF-CLEAN MODULES
0,2012-07-06,3218.00,9.37536,1,0,0,0,0,0,0,0,0,0,0,0,0.0
1,2012-07-07,3633.00,9.37536,1,0,0,0,0,0,0,0,0,0,0,0,0.0
2,2012-07-08,4377.00,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0
3,2012-07-09,4625.00,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0
4,2012-07-10,648.00,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,2018-12-27,4129.29,5.74700,1,0,1,0,0,0,0,0,0,0,0,0,971.0
2366,2018-12-28,4266.50,5.68900,0,0,1,0,0,0,0,0,0,0,0,0,0.0
2367,2018-12-29,3732.03,5.83400,1,0,1,0,0,0,0,0,0,0,0,0,720.0
2368,2018-12-30,4063.88,5.58400,0,1,0,0,0,0,0,0,0,0,0,0,0.0


In [55]:
#data.holiday.unique()

In [56]:
target = 'Total Generation (KWH)'

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2370 entries, 0 to 2369
Data columns (total 16 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Date                                     2370 non-null   object 
 1   Total Generation (KWH)                   2370 non-null   float64
 2   Insolation                               2370 non-null   float64
 3   Grid Failure                             2370 non-null   int64  
 4   Inverter                                 2370 non-null   int64  
 5   Module Cleaning                          2370 non-null   int64  
 6   Rainy day                                2370 non-null   int64  
 7   No Module Cleaning                       2370 non-null   int64  
 8   Transformer replacement and maintenance  2370 non-null   int64  
 9   Cable and Fuse maintenance               2370 non-null   int64  
 10  Plant Shutdown                           2370 no

In [58]:
# plt.figure(figsize=(16,9))
# fig = px.line(data, x=data.Date, y=data[target], title='total power genration')
# fig.update_xaxes(rangeslider_visible=True)
# fig.show() 


In [59]:
data.describe()

,Total Generation (KWH),Insolation,Grid Failure,Inverter,Module Cleaning,Rainy day,No Module Cleaning,Transformer replacement and maintenance,Cable and Fuse maintenance,Plant Shutdown,Internet,Battery,Cloudy,Module Cleaning by rains,NO-OFF-CLEAN MODULES
count,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.0,2.370000e+03
mean,3968.274734,9.375360,0.362025,0.102532,0.336287,0.018565,0.036287,0.008017,0.027848,0.000422,0.025316,0.011392,0.169198,0.0,1.133006e+04
std,1130.524750,91.909923,0.480688,0.303410,0.472538,0.135013,0.187043,0.089196,0.164572,0.020541,0.157118,0.106148,0.375006,0.0,5.406768e+05
min,0.000000,0.451000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00
25%,3347.177500,5.380250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00
50%,4198.810000,6.255000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00
75%,4785.212500,9.375360,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,6.040000e+02
max,6147.740000,4240.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,2.632180e+07


In [60]:
data.drop_duplicates(subset=['Date'], keep=False,inplace=True)
data.drop(['Date'],axis=1,inplace=True)

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2370 entries, 0 to 2369
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Total Generation (KWH)                   2370 non-null   float64
 1   Insolation                               2370 non-null   float64
 2   Grid Failure                             2370 non-null   int64  
 3   Inverter                                 2370 non-null   int64  
 4   Module Cleaning                          2370 non-null   int64  
 5   Rainy day                                2370 non-null   int64  
 6   No Module Cleaning                       2370 non-null   int64  
 7   Transformer replacement and maintenance  2370 non-null   int64  
 8   Cable and Fuse maintenance               2370 non-null   int64  
 9   Plant Shutdown                           2370 non-null   int64  
 10  Internet                                 2370 no

In [62]:
data.columns

Index(['Total Generation (KWH)', 'Insolation', 'Grid Failure', 'Inverter',
       'Module Cleaning', 'Rainy day', 'No Module Cleaning',
       'Transformer replacement and maintenance', 'Cable and Fuse maintenance',
       'Plant Shutdown', 'Internet', 'Battery', 'Cloudy',
       'Module Cleaning by rains', 'NO-OFF-CLEAN MODULES'],
      dtype='object')

In [63]:
data.select_dtypes('object').columns

Index([], dtype='object')

In [64]:
for i in data.select_dtypes('object').columns:
  le = LabelEncoder().fit(data[i])
  data[i] = le.transform(data[i])

In [65]:
data

,Total Generation (KWH),Insolation,Grid Failure,Inverter,Module Cleaning,Rainy day,No Module Cleaning,Transformer replacement and maintenance,Cable and Fuse maintenance,Plant Shutdown,Internet,Battery,Cloudy,Module Cleaning by rains,NO-OFF-CLEAN MODULES
0,3218.00,9.37536,1,0,0,0,0,0,0,0,0,0,0,0,0.0
1,3633.00,9.37536,1,0,0,0,0,0,0,0,0,0,0,0,0.0
2,4377.00,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0
3,4625.00,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0
4,648.00,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,4129.29,5.74700,1,0,1,0,0,0,0,0,0,0,0,0,971.0
2366,4266.50,5.68900,0,0,1,0,0,0,0,0,0,0,0,0,0.0
2367,3732.03,5.83400,1,0,1,0,0,0,0,0,0,0,0,0,720.0
2368,4063.88,5.58400,0,1,0,0,0,0,0,0,0,0,0,0,0.0


In [66]:
#data = data.drop(target,axis=1)
validate = data[data.drop(target,axis=1).columns].tail(10)
data.drop(data.tail(10).index,inplace=True)

In [67]:
validate

,Insolation,Grid Failure,Inverter,Module Cleaning,Rainy day,No Module Cleaning,Transformer replacement and maintenance,Cable and Fuse maintenance,Plant Shutdown,Internet,Battery,Cloudy,Module Cleaning by rains,NO-OFF-CLEAN MODULES
2360,5.335,1,0,1,0,0,0,0,0,0,0,0,0,720.0
2361,5.482,1,0,0,0,0,0,0,0,0,0,0,0,0.0
2362,5.171,1,0,1,0,0,0,0,0,0,0,0,0,1122.0
2363,5.276,0,0,1,0,0,0,0,0,0,0,0,0,630.0
2364,5.219,0,0,1,0,0,0,0,0,0,0,0,0,814.0
2365,5.747,1,0,1,0,0,0,0,0,0,0,0,0,971.0
2366,5.689,0,0,1,0,0,0,0,0,0,0,0,0,0.0
2367,5.834,1,0,1,0,0,0,0,0,0,0,0,0,720.0
2368,5.584,0,1,0,0,0,0,0,0,0,0,0,0,0.0
2369,5.635,1,0,1,0,0,0,0,0,0,0,0,0,1122.0


In [68]:
def custom_ts_multi_data_prep(dataset, target, start, end, window, horizon):
    X = []
    y = []
    start = start + window
    if end is None:
        end = len(dataset) - horizon

    for i in range(start, end):
        indices = range(i-window, i)
        X.append(dataset[indices])

        indicey = range(i+1, i+1+horizon)
        y.append(target[indicey])
    return np.array(X), np.array(y)

In [69]:
data.columns

Index(['Total Generation (KWH)', 'Insolation', 'Grid Failure', 'Inverter',
       'Module Cleaning', 'Rainy day', 'No Module Cleaning',
       'Transformer replacement and maintenance', 'Cable and Fuse maintenance',
       'Plant Shutdown', 'Internet', 'Battery', 'Cloudy',
       'Module Cleaning by rains', 'NO-OFF-CLEAN MODULES'],
      dtype='object')

In [70]:
#X_scaler = MinMaxScaler()
#Y_scaler = MinMaxScaler()
#X_data = X_scaler.fit_transform(data[data.drop(target,axis=1).columns])
#Y_data = Y_scaler.fit_transform(data[target])

# data = data.drop(['Insolation', 'Plant Shutdown'], axis=1)
# X_data = data[data.drop(target,axis=1).columns]
# X_data = data[['Grid Failure', 'Inverter', 'Module Cleaning', 'Cloudy']]
X_data = data['Grid Failure']
Y_data = data[target]

In [71]:
hist_window = 48
horizon = 10
TRAIN_SPLIT = 30000
x_train, y_train = custom_ts_multi_data_prep(X_data, Y_data, 0, TRAIN_SPLIT, hist_window, horizon)
x_vali, y_vali = custom_ts_multi_data_prep(X_data, Y_data, TRAIN_SPLIT, None, hist_window, horizon)

KeyError: '[2360] not in index'

In [ ]:
x_train

In [ ]:
print ('Multiple window of past history\n')
print(x_train[0])
print ('\n Target horizon\n')
print (y_train[0])

In [ ]:
batch_size = 256
buffer_size = 150

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(buffer_size).batch(batch_size).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_vali, y_vali))
val_data = val_data.batch(batch_size).repeat()

In [ ]:
lstm_model = tf.keras.models.Sequential([
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, return_sequences=True), 
                               input_shape=x_train.shape[-2:]),
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(units=horizon),
])
lstm_model.compile(optimizer='adam', loss='mse')


In [ ]:
lstm_model.summary()

In [ ]:
model_path = 'Bidirectional_LSTM_Multivariate.h5'
early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

In [ ]:
history = lstm_model.fit(train_data,epochs=150,steps_per_epoch=100,validation_data=val_data,validation_steps=50,verbose=1,callbacks=callbacks)

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'validation loss'])
plt.show()

In [ ]:
data.columns

In [ ]:
  data_val = X_scaler.fit_transform(data[['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main',
       'weather_description', 'traffic_volume']].tail(48))
  val_rescaled = data_val.reshape(1, data_val.shape[0], data_val.shape[1])

In [ ]:
pred = lstm_model.predict(val_rescaled)
pred

In [ ]:
pred_Inverse = Y_scaler.inverse_transform(pred)

In [ ]:
pred_Inverse

In [ ]:
def timeseries_evaluation_metrics_func(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')

In [ ]:
pred_Inverse[0]

In [ ]:
timeseries_evaluation_metrics_func(validate['traffic_volume'],pred_Inverse[0])

In [ ]:
plt.figure(figsize=(16,9))
plt.plot( list(validate['traffic_volume']))
plt.plot( list(pred_Inverse[0]))
plt.title("Actual vs Predicted")
plt.ylabel("Traffic volume")
plt.legend(('Actual','predicted'))
plt.show()